# Foreword

The most important requirement of a code is not its switftness. It is that it provides the right answer. Then, and only then is it time to optimize if its performance proves unsatisfactory for your needs. The strongest gains are often to be found not in the few tips and tricks presented here but in the structure of the code and its high level algorithms, points that are well beyond the scope of this short tutorial. It is therefore important to consider such aspects before trying to optimize the details of the implementation. Yet, as we will see, these can nevertheless yield nice speedups for a minimal investment.

These tips and tricks mostly come from my experience developing [Cigale](http:://cigale.lam.fr). I do not claim they are original (they certainly are not) and I would be more than happy to add new ideas to this notebook. Come talk to me ot send me a word at mederic.boquien@uantof.cl.

Note: you will likely need a recent version of Python 3 to run all of the examples below.

---

# A couple of useful tools

Two of the main traps when attempting to optimize a program are: 1. to assume you know what the bottlenecks of your code are, and 2. to assume that a method is faster than another. Do **not** assume. It is not unlikely that you are incorrect in your assumptions. Profile, *profile*, **profile**. There are numerous great tools around to do just that. I especially recommend a couple of them that have proven tremendously useful for me: [gprof2dot](https://github.com/jrfonseca/gprof2dot) and [line_profiler](https://github.com/rkern/line_profiler). A small limitation though: these tools do not work properly when running a parallel code.

## gprof2dot

This is a script that creates call graphs, shows what are the hot paths in your program, and how much time is spent in the most time-consuming functions so you can identify the bottlenecks and know where to focus your efforts. Its usage is simple:
```bash
python -m profile -o output.pstats path/to/your/script.py arg1 arg2
gprof2dot -f pstats output.pstats | dot -Tsvg -o output.svg
```
This generates an SVG file of your call graph, which can be displayed, for instance, with Chrome or Firefox. Bonus, function names are searchable as in a web page. Note: if you cannot find the dot program, it comes with [Graphviz](http://www.graphviz.org/).

## line\_profiler

This is a script that display how much time is spent on each instruction in specific functions. To use it simply add the @profile decorator to the functions you want to profile and run your script as follows:
```bash
kernprof -l path/to/your/script arg1 arg2
python -m line_profiler script.py.lprof
```
This will display the time spent in each function and for each line, the number of hits, the time spent per hit, the percentage of time spent on a line compared to the total time, and the content of the line.

---

In [ ]:
import numpy as np

---

# Avoid loops

Let's start by stating the obvious: python loops are slow and should be avoided as much as possible. As an example, let's compute the sum of the *n* first natural integers first with a pure python loop, then without a loop but still in pure python, and finally using Numpy.

In [ ]:
def loop_python(n):
    intsum = 0
    for i in range(1, n+1):
        intsum += n
    return intsum

def noloop_python(n):
    return sum(range(1, n+1))

def noloop_numpy(n):
    return np.sum(np.arange(1, n+1))

n = 1000000
%timeit loop_python(n)
%timeit noloop_python(n)
%timeit noloop_numpy(n)

Sometimes loops are hard to avoid. In that case it is better to have few iterations on large arrays rather than many iterations on small arrays.

In [ ]:
def compute_sum(arr):
    arrsum = np.zeros(arr.shape[1])
    for i in range(arr.shape[0]):
        arrsum += np.sum(arr[i, :])
    return np.sum(arrsum)

x = np.random.rand(1000000).reshape(10, 100000)
y = np.random.rand(1000000).reshape(100000, 10)

%timeit compute_sum(x)
%timeit compute_sum(y)

**Conclusion: avoid loops whenever possible and when it is not possible ensure that each iteration operates on as many elements as possible.**

---

# Stay in place

Two types of operations are possible on an array: in-place or not in-place. When an operation is in-place it means the operation is carried out directly in the array. Otherwise it goes through intermediate arrays whose allocation can be costly and kill the cache of the CPU.

You may be familiar with the in-place operations as they come with handy shorthands: +=, -=, \*=, /=, or \*\*= for the most common ones. As they avoid the aforementioned array allocation, they happen to be quite a bit faster than the alternative.

In [ ]:
def inplace_sum(arr):
    arr += 0.

def notinplace_sum(arr):
    arr = arr + 0.

%timeit inplace_sum(x)
%timeit notinplace_sum(x)

Even for complex operations, it may be worthwhile to decompose them in terms of elementary in-place operations

In [ ]:
def inplace_norm(arr):
    vmax = np.max(arr)
    vmin = np.min(arr)
    arr -= vmax
    arr *= 1. / (vmax - vmin)

def notinplace_norm(arr):
    vmax = np.max(arr)
    vmin = np.min(arr)
    arr = (arr - vmax) * (1. / (vmax - vmin))

%timeit inplace_norm(x)
%timeit notinplace_norm(x)

**Conclusion: avoid complex formulas that require the creation of temporary arrays. Series of in-place operations are likely faster (though a bit less readable sometimes).**

---

# Do not get fancy unless needed

Fancy indexing is really a nice and convenient way to make complex selections on arrays. However fancy indexing can be quite slow and an array indexed that way generates a copy. When possible, it is better to use slices as they only create a view of the array with just the right elements. Let's compare the two approaches:

In [ ]:
x = np.random.rand(10000000)
z = np.tile(np.arange(10.), 1000000)

w = np.where(z == 3.)

print('Are the two arrays the same? {}'.format(np.all(x[w] == x[3::10])))
print('Do the two arrays share memory? {}'.format(np.may_share_memory(x[w], x[3::10])))

%timeit x[w]
%timeit x[3::10]

Note that sometimes it is convenient to create slices programmatically. This can be done easily.

In [ ]:
sl = slice(3, None, 10)

print('Are the two arrays the same? {}'.format(np.all(x[sl] == x[3::10])))
print('Do the two arrays share memory? {}'.format(np.may_share_memory(x[sl], x[3::10])))

**Conclusion: it is faster and saves memory to use slices. Beware if you modify the resulting array though.**

---

# <s>Divide</s>Multiply and conquer
Modern CPU have become good at handling floats natively with fairly good efficiency. However not all basic operations on floats have been created equal and some are faster than others. Let's find out which.

In [ ]:
x = np.random.rand(1000000)

%timeit x + np.pi
%timeit x - np.pi
%timeit x * np.pi
%timeit x / np.pi

**Conclusion: avoid dividing arrays, prefer multiplications when possible. Note that CPUs are good at dividing by (powers of) 2. In that case the speed difference is much smaller.**

---

# Memory matters
How we store multi-dimensional arrays can have an important effect on the final performance. Let's start with a small experiment.

In [ ]:
x = np.arange(5000**2).reshape(5000, 5000)

In [ ]:
%timeit np.sum(x, axis=0)
%timeit np.sum(x, axis=1)

Operating on the second axis appears faster than operating on the first axis. Each time we operate on the same number of elements so it does not relate to the size of the array. This is actually due to how the array is stored in memory. Numpy arrays are C-ordered. This means that the last dimension is the one that varies the fastest. In effect x is stored linearly in memory as follows: 1 2 … 4998 4999 5000 5001 … 9998 9999 10000 10001 … […] … 24999998 24999999. So when we operate on the second axis, numpy operates on contiguous arrays: 1 2 … 4998 4999, then 5000 5001 … 9998 9999, etc. CPU are very good at reading contiguous data as they can read the data in advance (prefetch) and can cache them efficiently. When we operate on the first axis, things go very differently though. Numpy operates on non-contiguous arrays: 0 5000 … 24990000 24995000 then 1 5001 … 24990001 24995001. As these values are stored in very different locations in memory, the CPU cannot read the data ahead of being required (note that it can if the stride is not too large but here the stride is ~40kB, which is too large even for modern CPUs).

**Conclusion: whenever you can, have arrays shapes that ensure that you operate on the last axis as it is the fastest-varying index.**

---

# Cache

The best optimization you can do when computing a quantity is not having to compute it at all. This is especially true if you often call a given function with the same arguments. In that case it would be smart to just keep the results in memory. This can be done using the @lru_cache decorator provided in the functools module.

In [ ]:
from functools import lru_cache
import numpy as np

def nocache_sum(n):
    return np.sum(np.arange(1, n+1))

@lru_cache()
def cache_sum(n):
    return np.sum(np.arange(1, n+1))

n = 1000000
%timeit nocache_sum(n)
%timeit cache_sum(n)

While this is convenient, it does not cover the case where the parameters are not hashable, such as Numpy arrays. Under these circumstances the cache has to be done manually. A very naïve (and quite inefficient but that is just for the sake of illustration) cache implementation for a Numpy array would be the following:

In [ ]:
def cache(func):
    cache = {}
    def helper(arr):
        key = hash(arr.tobytes()) # This is slow. It is only worthwhile if the cached function is very slow 
        if key not in cache:
            cache[key] = func(arr)
        return cache[key]
    return helper

def nocache_sumexp(arr):
    return np.sum(np.exp(arr))
    
@cache
def cache_sumexp(arr):
    return np.sum(np.exp(arr))

x = np.random.rand(1000000)
%timeit nocache_sumexp(x)
%timeit cache_sumexp(x)

**Conclusion: caching frequently called and/or expensive functions can save you a tremendous amount of time at the cost of some RAM. The balance is yours to strike.**

---

# Eliminate Numpy overheads

Maybe you are spending a large part of your time repeatedly calling the same Numpy function. Numpy tends to be quite careful when processing the arguments, making sure everything is alright, checking various corner cases, etc. This is a good thing and we want Numpy to be cautious. However in a very well controled environment where we are always in the same specific conditions, the overhead coming from numpy checks may come at such a cost that they want to be avoided. Let's take the Numpy trapezoidal rule integration function for instance. A large majority of the lines of code is taken by these checks.

In [ ]:
np.source(np.trapz)

Let's say we are in the simple case of 1D integration and we are certain we pass 1D Numpy arrays. The implementation could be simplified to eliminate the overhead, which would yield a nice speedup.

In [ ]:
def trapz_v1(y, x):
    return (np.diff(x) * (y[1:]+y[:-1])).sum() / 2.

x = np.logspace(1, 2, 1000)
y = np.random.rand(1000)
%timeit np.trapz(y, x)
%timeit trapz_v1(y, x)

**Conclusion: the overhead of Numpy functions can be largely eliminated when the call conditions (type, dimension, etc.) are perfectly under control.**

---

# Going beyond

Naturally these various tips and tricks do not come in isolation and can easily be combined. If we continue expanding from the trapz example, considering a case where we call it millions or billions of times, assuming the x grid is the same, it would be interesting to maybe cache dx using the naïve cache implementation earlier. However the computation of the hash is rapidly a bottleneck so a smarter algorithm would be needed. In any case, we saw that a multiplication is faster than a division so we can gain a few cycles at no cost multiplying by 0.5 rather than by dividing by 2. But if we pay close attention, the computation of the integral can be rewritten as a dot product. Dot products can be very fast with Numpy as they can use highly optimized linear algebra libraries such as [OpenBLAS](http://www.openblas.net/) for instance. They can be multi-threaded, wich gives a nice bonus when your program is not already parallel. To check whether you have optimized libraries you can simply look at the configuration of Numpy.

In [ ]:
np.__config__.show()

If neither OpenBLAS, ATLAS, nor MKL are available, it is likely that you only have the slower unoptimized library installed. In any case, even then using the dot product yields nice improvements. 

In [ ]:
def trapz_v2(y, x):
    return np.dot(np.diff(x), y[1:]+y[:-1]) * .5

x = np.logspace(1, 2, 1000)
y = np.random.rand(1000)
%timeit np.trapz(y, x)
%timeit trapz_v1(y, x)
%timeit trapz_v2(y, x)

**Conclusion: with a focused effort on the hotspots, code can be made much faster with just a few tips. With habit some of these tips can become second nature and help write naturally efficient code.**